This script aims to:
### 1. Improve the quality of feedback received from customersof SaaS platform for marketing automation & customer engagement) by ensuring that the feedback delivered by the customer service team to the product team meets high standards. This includes providing the necessary context, a clear description of the issues, and proper categorization based on the application area being addressed.

### 2. Analyze the collected feedback to identify the most significant user problems.
This involves categorizing the feedback under relevant initiatives, and determining the frequency and intensity of specific issues or needs expressed by users.

- - -
- - -

## STEP 1. Improving quality of customer feedback with OpenAI assistance

### 1.1 Agent, definitions & function preparation

In [ ]:
import pandas as pd
from textwrap import dedent
from openai import OpenAI
import os

# Konfiguracja OpenAI API
api_key = "ADD_YOUR_API_KEY_HERE" # ADD_YOUR_API_KEY_HERE
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))

MODEL = "gpt-4o-2024-08-06"

# Agent
agent = '''
    You are SALESmanago - desktop platform helping with marketing automation and customer engagement for medium-sized eCommerces globally - expert.
    Your job is to process user feedbacks delivered through the Customer Service Team to enhance their quality, creating user stories, describing pain points, and categorizing them by adding labels.
    To boost creating detailed descriptions, you use knowledge about work in SALESmanago and general marketer's perspective gathered through support articles (https://support.salesmanago.com/*).
    Also, familiarize yourself with growth module & SALESmanago pricing package: https://www.salesmanago.com/info/growth-framework.htm
'''

# Funkcja przetwarzająca feedback
def process_feedback(issue_key, feedback_description):
    prompt = f'''
        Acting as a SALESmanago user, improve the quality of feedback provided by the Customer Success and Support team.
        
        Here is the provided feedback:
        Issue Key: {issue_key}
        Description: {feedback_description}
        
        Based on this feedback:
        - Develop a user story aligned with Design Thinking principles, including: Who (persona), What (need/goal) & Why (user context and objective).
        - Highlight the top 3-5 detailed user's pain points in relation to system performance and overall client experience.
        - Assign to the respective module(s) [Modules: Starter, Audiences, Channels, Website_Experience, Recommendations, Automation, Insights, Whole_Platform] 
        
        Return your response in the following format:
        User Story: [insert user story]
        Pain Points: [insert pain points]
        Module: [insert Module]
    '''
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": dedent(agent)},
                {"role": "user", "content": dedent(prompt)}
            ],
            max_tokens=500
        )
        
        result = response.choices[0].message.content
        
        # Rozdziel odpowiedź na User Story, Pain Points i Module
        user_story, pain_points, module = "", "", ""
        if "User Story:" in result and "Pain Points:" in result and "Module:" in result:
            parts = result.split("Pain Points:")
            user_story_part = parts[0].replace("User Story:", "").strip()
            
            # Dalsze dzielenie na Pain Points i Module
            pain_points_part = parts[1].split("Module:")[0].strip()
            module_part = parts[1].split("Module:")[1].strip()
            
            user_story = user_story_part
            pain_points = pain_points_part
            module = module_part
        
        return user_story, pain_points, module
    except Exception as e:
        print(f"Błąd podczas przetwarzania Issue Key: {issue_key} - {str(e)}")
        return "Error processing feedback", "Error processing feedback", "Error processing module"


### 1.2 Code implementation, starting with reading the feedback file, and saving results in csv as well

In [ ]:
# Wczytanie danych z pliku CSV
input_file = 'ALL_FEED_description.csv'
output_file = 'Processed_Feedbacks_with_Modules.csv'

data = pd.read_csv(input_file)

# Dodanie nowych kolumn do przechowywania wyników
data['User Story'] = ""
data['Pain Points'] = ""
data['Module'] = ""

# Eksport nagłówków do pliku wynikowego
data.head(0).to_csv(output_file, index=False)

# Przetwarzanie każdego wiersza
total_rows = len(data)
for index, row in data.iterrows():
    issue_key = row['Issue key']
    feedback_description = row['Description']
    
    print(f"Przetwarzanie {index + 1}/{total_rows}: Issue Key {issue_key}")
    
    user_story, pain_points, module = process_feedback(issue_key, feedback_description)
    data.at[index, 'User Story'] = user_story
    data.at[index, 'Pain Points'] = pain_points
    data.at[index, 'Module'] = module

    # Zapis przetworzonego wiersza do pliku wynikowego
    with open(output_file, 'a', encoding='utf-8', newline='') as f:
        data.iloc[[index]].to_csv(f, index=False, header=False)
    
    print(f"Zakończono {index + 1}/{total_rows}: Wyniki zapisane.")

print(f"Przetwarzanie zakończone. Wyniki zapisane w pliku: {output_file}")

#### + Cleaning 'Modules' column

In [8]:
import pandas as pd

# Lista poprawnych modułów
VALID_MODULES = [
    "Audiences", "Channels", "Recommendations", "Website_Experience",
    "Automation", "Whole_Platform", "Insights", "Starter"  
]

# Funkcja do czyszczenia modułów
def clean_module_column(data, column_name='Module'):
    def clean_modules(module_entry):
        if not isinstance(module_entry, str):
            return None
        # Rozdziel moduły na podstawie przecinków lub innych separatorów
        modules = [m.strip() for m in module_entry.split(",")]
        # Zachowaj tylko poprawne moduły
        valid_modules = [m for m in modules if m in VALID_MODULES]
        return ", ".join(valid_modules) if valid_modules else None

    # Zastosuj funkcję czyszczącą do kolumny
    data[column_name] = data[column_name].apply(clean_modules)
    # Usuń wiersze z pustymi modułami
    data = data[data[column_name].notna()]
    return data

# Ładowanie danych
input_file = 'Processed_Feedbacks_with_Modules.csv'
data = pd.read_csv(input_file)

# Czyszczenie kolumny 'Module'
cleaned_data = clean_module_column(data, column_name='Module')

# Eksport wyczyszczonych danych
cleaned_file = 'Cleaned_Feedbacks.csv'
cleaned_data.to_csv(cleaned_file, index=False)

print(f"Cleaning complete. Cleaned data saved to: {cleaned_file}")


Cleaning complete. Cleaned data saved to: Cleaned_Feedbacks.csv


- - - 
## STEP 2. Analyzing collected feedbacks

### 2.1 Setting up new agent

In [70]:
import pandas as pd
from textwrap import dedent
from openai import OpenAI
import os
import re
import time

# Konfiguracja OpenAI API
api_key = "ADD_YOUR_API_KEY_HERE" # ADD_YOUR_API_KEY_HERE
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))

MODEL = "gpt-4o-2024-08-06"

# Agent
agent = '''
    You are an expert in SALESmanago - a leading desktop platform specializing in marketing automation and customer engagement for medium-sized eCommerce businesses globally.
    You are also a meticulous data analyst with deep expertise in cognitive science and CX/UX design principles.
    Your role is to process user feedback collected by the Customer Service Team, identifying the most critical, frequent, and high-priority user needs and suggestions that should be addressed first.
    Utilize your in-depth knowledge of SALESmanago's platform, its features, and customer workflows, as well as insights from support articles (https://support.salesmanago.com/*) and your design and CX expertise, to provide actionable and strategic recommendations for product improvements.
'''

In [72]:
# Funkcja przetwarzająca feedback
def analyze_feedback_group(group_name, feedback_table):
    prompt = f'''
        You are an expert in analyzing customer feedback. 
        Group the following feedbacks related to the '{group_name}' area into no more than 15 most common and crucial initiatives.
        Provide the results in the following structured format:
        ###Initiative_Start###
        Title: [Short title]
        Description: [Detailed description of the initiative]
        Sentiment: [Sentiment analysis: frustrating/exciting/low priority]
        Percentage: [Ratio of feedbacks related to this initiative to all feedbacks]
        Feedback IDs: [Comma-separated list of IDs of all related feedbacks]
        Requirements: [Bullet-point list of requirements to address the issue]
        ###Initiative_End###
        
        Feedback table:
        {feedback_table.to_csv(index=False)}
    '''
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": dedent(agent)},
                {"role": "user", "content": dedent(prompt)}
            ],
            #max_tokens=5000
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing feedback for group {group_name}: {e}")
        return ""

# Zabezpieczenie przed błędami API związanymi ze zbyt dużą częstotliwością zapytań
def analyze_feedback_group_with_retry(group_name, feedback_table, max_retries=6):
    retries = 0
    while retries < max_retries:
        try:
            return analyze_feedback_group(group_name, feedback_table)
        except Exception as e:
            print(f"Error: {e}. Retrying {retries + 1}/{max_retries}...")
            time.sleep(5)  # Czekaj 5 sekund przed ponowieniem
            retries += 1
    print(f"Max retries exceeded for module {group_name}. Skipping...")
    return ""


# Funkcja parsująca odpowiedź agenta
def parse_initiatives(text):
    initiatives = []
    blocks = re.findall(r"###Initiative_Start###(.*?)###Initiative_End###", text, re.DOTALL) # Separator inicjatyw

    for block in blocks:
        if block.strip():  # Pomijanie pustych bloków
            try:
                # Wyodrębnij dane z poszczególnych linii
                title = re.search(r"Title: (.+)", block)
                description = re.search(r"Description: (.+)", block)
                sentiment = re.search(r"Sentiment: (.+)", block)
                percentage = re.search(r"Percentage: (.+)", block)
                feedback_ids = re.search(r"Feedback IDs: (.+)", block)
                requirements = re.search(r"Requirements:(.+)", block, re.DOTALL)

                #formatted_requirements = requirements.group(1).replace('\n', ';') if requirements else "N/A"
    
                # Zabezpieczenie przed pustymi polami
                title = title.group(1).strip() if title else "N/A"
                description = description.group(1).strip() if description else "N/A"
                sentiment = sentiment.group(1).strip() if sentiment else "N/A"
                percentage = percentage.group(1).strip() if percentage else "0%"
                feedback_ids = feedback_ids.group(1).strip() if feedback_ids else "N/A"
                requirements = requirements.group(1).strip() if requirements else "N/A"

                # Dodaj inicjatywę do listy
                initiatives.append({
                    "Title": title,
                    "Description": description,
                    "Sentiment": sentiment,
                    "Percentage": percentage,
                    "Feedback IDs": feedback_ids,
                    "Requirements": requirements
                })
            except Exception as e:
                print(f"Unexpected error parsing block: {block}\n{e}")

    return initiatives
    

In [74]:
# Pliki wejściowe i wyjściowe
input_file = 'Cleaned_Feedbacks.csv'
output_file = 'Module_Initiatives_Analysis.csv'

# Lista do przechowywania wszystkich inicjatyw
all_initiatives = []

# Ładowanie danych wejściowych
data = pd.read_csv(input_file)

# Rozdzielanie wartości w kolumnie 'Module' na listy
data['Module'] = data['Module'].str.split(',')

# Rozwijanie danych tak, aby każdy feedback był przypisany do pojedynczego modułu
expanded_data = data.explode('Module')
expanded_data['Module'] = expanded_data['Module'].str.strip()  # Usunięcie zbędnych spacji

# Grupowanie feedbacków na podstawie modułów
grouped_data = expanded_data.groupby('Module')

# Tworzenie lub otwieranie pliku w trybie zapisu
if not os.path.exists(output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("Module,Title,Description,Sentiment,Percentage,Feedback IDs,Requirements\n")


# Przetwarzanie każdego modułu
for module, group in grouped_data:
    print(f"Processing module: {module}")
    feedback_table = group[['Issue key', 'Description']]

    try:
        # Analiza przez agenta
        analysis = analyze_feedback_group_with_retry(module, feedback_table)
        
        # Parsowanie inicjatyw
        initiatives = parse_initiatives(analysis)

        # Dodanie informacji o module do każdej inicjatywy
        for initiative in initiatives:
            initiative["Module"] = module
            all_initiatives.append(initiative)
        
        print(f"Finished processing module: {module}")
    except Exception as e:
        print(f"Error processing module {module}: {e}")

# Konwersja danych do DataFrame i zapis do pliku CSV
df = pd.DataFrame(all_initiatives)
df.to_csv(output_file, index=False, sep=';', quotechar='"', quoting=1, encoding='utf-8')

print(f"Analysis complete. Results saved to: {output_file}")



Processing module: Audiences
Finished processing module: Audiences
Processing module: Channels
Finished processing module: Channels
Processing module: Insights
Finished processing module: Insights
Processing module: Recommendations
Finished processing module: Recommendations
Processing module: Starter
Finished processing module: Starter
Processing module: Website_Experience
Finished processing module: Website_Experience
Processing module: Whole_Platform
Finished processing module: Whole_Platform
Analysis complete. Results saved to: Module_Initiatives_Analysis.csv
